In [8]:
import copy
import random
import Bio
from Bio import SeqIO
import numpy as np

In [2]:
def lookahead(start_position, seq):
    for k in range(start_position, len(seq)):
        if seq[k].upper() == seq[k]:
            break
    return k - start_position

In [3]:
with open("uniref90_hits.a3m", "r") as f:
    seqs = [str(s.seq) for s in SeqIO.parse(f, "fasta")]

In [4]:
insert_positions = {}
for s in seqs:
    counter = 0
    i = 0
    if "X" in s or "B" in s:
        continue
    while i < len(s):
        i += 1
        if i >= len(s):
            break
        letter = s[i]
        if letter.upper() != letter:
            num_inserts = lookahead(i, s)
            if counter in insert_positions:
                if num_inserts > insert_positions[counter]:
                    insert_positions[counter] = num_inserts
            else:
                insert_positions[counter] = num_inserts
            i += num_inserts
        counter += 1

In [5]:
clean_seqs = []
for s in seqs:
    clean_seq = [s[0]]
    i, counter = 0, 0
    if "X" in s or "B" in s:
        continue
    while i < len(s):
        i += 1
        if i >= len(s):
            break
        letter = s[i]
        if letter.upper() != letter:
            num_inserts = lookahead(i, s)
            num_blanks = insert_positions[counter] - num_inserts
            clean_seq.append(s[i:i+num_inserts].upper())
            clean_seq.append("".join(["-" for j in range(num_blanks)]))
            i += num_inserts
        elif counter in insert_positions:
            clean_seq.append("".join(["-" for j in range(insert_positions[counter])]))
        clean_seq.append(s[i])
        counter += 1
    clean_seqs.append("".join(clean_seq))

In [7]:
with open("converted_seqs.txt", 'w+') as f:
    for s in clean_seqs:
        _ = f.write(s)
        _ = f.write("\n")